In [ ]:
import hashlib
import torch

In [ ]:
from pathlib import Path
from typing import List, Optional, Tuple, Union

import numpy as np
from bioimageio.spec.model.v0_5 import (
    ArchitectureFromLibraryDescr,
    ArchitectureFromFileDescr,
    Author,
    AxisBase,
    AxisId,
    BatchAxis,
    ChannelAxis,
    EnvironmentFileDescr,
    FileDescr,
    FixedZeroMeanUnitVarianceDescr,
    FixedZeroMeanUnitVarianceKwargs,
    Identifier,
    InputTensorDescr,
    ModelDescr,
    OutputTensorDescr,
    PytorchStateDictWeightsDescr,
    SpaceInputAxis,
    SpaceOutputAxis,
    TensorId,
    Version,
    WeightsDescr,
)

Example:
[https://github.com/bioimage-io/spec-bioimage-io/blob/main/example/load_model_and_create_your_own.ipynb](https://github.com/bioimage-io/spec-bioimage-io/blob/main/example/load_model_and_create_your_own.ipynb)

In [ ]:
# README.md file
doc_md = "path/to/readme/file.md"

In [ ]:
# model input
input_path = "path/to/input/sample.npy"
input_sample = np.load(input_path)
# building axes
in_axes = [BatchAxis()]  # batch is always there!
# channel
in_axes.append(
    ChannelAxis(channel_names=[Identifier("channel")])
)
# spatial dims
in_axes.append(
    SpaceInputAxis(id=AxisId("y"), size=input_sample.shape[2])
)
in_axes.append(
    SpaceInputAxis(id=AxisId("x"), size=input_sample.shape[3])
)
# input descriptor
input_descr = InputTensorDescr(
    id=TensorId("input"),
    axes=in_axes,
    test_tensor=FileDescr(source=input_path),
    preprocessing=[
        FixedZeroMeanUnitVarianceDescr(
            kwargs=FixedZeroMeanUnitVarianceKwargs(mean=mu, std=std)
        )
    ],
)


In [ ]:
# model output
output_path = "path/to/input/sample.npy"
output_sample = np.load(output_path)
# building axes
out_axes = [BatchAxis()]  # batch is always there!
# channel
out_axes.append(
    ChannelAxis(channel_names=[Identifier("channel")])
)
# spatial dims
out_axes.append(
    SpaceOutputAxis(id=AxisId("y"), size=output_sample.shape[2])
)
out_axes.append(
    SpaceOutputAxis(id=AxisId("x"), size=output_sample.shape[3])
)
# input descriptor
output_descr = OutputTensorDescr(
    id=TensorId("prediction"),
    axes=out_axes,
    test_tensor=FileDescr(source=output_path),
    preprocessing=[
        FixedZeroMeanUnitVarianceDescr(
            kwargs=FixedZeroMeanUnitVarianceKwargs(mean=inv_mu, std=inv_std)
        )
    ],
)


[https://bioimage-io.github.io/spec-bioimage-io/bioimageio/spec/model/v0_5.html#ArchitectureDescr](https://bioimage-io.github.io/spec-bioimage-io/bioimageio/spec/model/v0_5.html#ArchitectureDescr)

In [ ]:
# model arch & weights
model_src_file = "model.py"
# params to instantiate the model (pass to __init__)
model_kwargs = {}
model_sha256 = hashlib.sha256(model_src_file)

arch_descr = ArchitectureFromFileDescr(
    source=model_src_file,
    sha256=model_sha256,
    callable="ModelClassName",
    kwargs=model_kwargs
)
# architecture_descr = ArchitectureFromLibraryDescr(
#     import_from="careamics.models",
#     callable=f"{config.algorithm_config.model.architecture}",
#     kwargs=config.algorithm_config.model.model_dump(),
# )

pytorch_version = Version(torch.__version__)
weights_path = "path/to/weights.pth"
env_path = "path/to/env.yml"

weights_descr = WeightsDescr(
    pytorch_state_dict=PytorchStateDictWeightsDescr(
        source=weights_path,
        architecture=arch_descr,
        pytorch_version=Version(pytorch_version),
        dependencies=EnvironmentFileDescr(source=env_path),
    ),
)

In [ ]:
# authors
au1 = Author(
    name="Ashesh",
    email="",
    affiliation="",
    orcid="",
    github_user=""
)
au2 = Author(
    name="John Doe",
    email="",
    affiliation="",
    orcid="",
    github_user=""
)

authors = [au1, au2]

In [ ]:
# model
model_descr = ModelDescr(
    name="name",
    authors=authors,
    description="description",
    documentation=doc_md,
    inputs=[input_descr],
    outputs=[output_descr],
    tags=["tag1", "tag2"],
    # covers=["cover1.png"],
    links=[
        "https://github.com/CAREamics/careamics",
        "https://careamics.github.io/latest/",
    ],
    license="BSD-3-Clause",
    version="0.1.0",
    weights=weights_descr,
    # attachments=[FileDescr(source="")],
    cite="",
    config={  # conversion from float32 to float64 creates small differences...
        "bioimageio": {
            "test_kwargs": {
                "pytorch_state_dict": {
                    "decimals": 2,  # ...so we relax the constraints on the decimals
                }
            }
        }
    },
)

In [ ]:
model_descr.validation_summary.display()

In [ ]:
from bioimageio.core import test_model

summary = test_model(model_descr)
summary.display()

In [ ]:
from bioimageio.spec import save_bioimageio_package

save_bioimageio_package(model_descr, output_path=Path("my_model.zip"))